In [1]:
from newspaper import Article
from transformers import AutoConfig, AutoTokenizer, AutoModel
from summarizer import Summarizer
import pandas as pd
from transformers import PegasusForConditionalGeneration, AutoTokenizer
import torch

/opt/bitnami/miniconda/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/opt/bitnami/miniconda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip install pandas

In [4]:
url = "https://www.npr.org/2022/11/12/1136205315/musk-twitter-bankruptcy-how-likely"
article = Article(url)
article.download()
article.parse()
txt = article.txt

In [5]:
article.download()

In [6]:
article.parse()

In [7]:
article.authors

[]

In [9]:
txt = article.text

In [ ]:
from summarizer import Summarizer

In [ ]:
def generate_summary_from_text(text):

    model = Summarizer()
    result = model(text, ratio=0.2)

    return result

In [ ]:
print(generate_summary_from_text(article.text))

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
from summarizer import Summarizer

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained(pretrained_model_name_or_path="indobenchmark/indobert-base-p1")
custom_config.output_hidden_states = True
custom_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="indobenchmark/indobert-base-p1")
custom_model = AutoModel.from_pretrained("./IndoBERT/indobert-base-p1", config=custom_config)

In [15]:
df = pd.read_csv("gdelt.csv")
urls = df["URLs"]

In [16]:
first_ten = urls[0:10]

In [18]:
first_ten[5]

'https://news.yahoo.com/waiting-2-weeks-address-twitter-110801142.html?guccounter=1&guce_referrer=aHR0cHM6Ly9hcGkuZ2RlbHRwcm9qZWN0Lm9yZy8&guce_referrer_sig=AQAAANzLNVzWaIFvpMpBKIYllRx9I_peg90-RHcOeBg20QHqtoj1wYpiTO27C0ZajBaXrPi1CD1WoiBG_mIxn1ZqXDK2Pr0M8h8vNR3HFEM9UFhuI_gD8KWeRlS55s8U7H3PXKxBHs_RFTqVVd-B36YpPzanXhpGpqyTJPHjV5oN-QPl'

In [ ]:
results = {}

In [ ]:
for url in first_ten:
    result = ""
    try:
        article = Article(url)
        article.download()
        article.parse()
        result = generate_summary_from_text(article.text)
        results[url] = result
    except:
        results[url] = "FAILED"
    

In [ ]:
list(results.keys())

In [ ]:
list(results.values())

In [ ]:
df_final  = pd.DataFrame(columns=["url", "summary"])

In [14]:
df_final["url"] = list(results.keys())

NameError: name 'results' is not defined

In [ ]:
txt = article.text

In [ ]:
df_final["summary"] = list(results.values())

In [ ]:
df_final.to_csv("summaries_2.csv")

In [32]:
all_urls = urls

In [53]:
results = {}

In [ ]:
url = "https://www.npr.org/2022/11/12/1136205315/musk-twitter-bankruptcy-how-likely"
article = Article(url)
article.download()
article.parse()
txt = article.txt

In [57]:
for url in all_urls:
    try:
        article = Article(url)
        article.download()
        article.parse()
        txt = article.text
        # You can chose models from following list
        # https://huggingface.co/models?sort=downloads&search=google%2Fpegasus
        model_name = 'google/pegasus-multi_news'
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
        batch = tokenizer(txt, truncation=True, padding='longest', return_tensors="pt").to(device)
        translated = model.generate(**batch)
        tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        results[url] = tgt_text
    except:
        results[url] = "FAILURE"

Downloading: 100%|██████████| 2.28G/2.28G [03:21<00:00, 11.3MB/s]
/opt/bitnami/jupyterhub-singleuser/.local/lib/python3.7/site-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [58]:
results_clean = {}

In [59]:
for key, value in results.items():
    
    results_clean[key] = (value[0].replace("<n>", "\n"))

In [60]:
results_clean

{'https://www.npr.org/2022/11/12/1136205315/musk-twitter-bankruptcy-how-likely': '– Elon Musk says he\'s open to bailing out Twitter if the social media company doesn\'t start making more money, and he\'s apparently got plenty of cash on hand to do just that. The problem is that Twitter is saddled with $13 billion in debt—seven times the company\'s projected earnings for 2022—and only has $632 million in cash on hand last year, NPR reports. "It\'s hypothetically possible that he could use more of his Tesla stock to bail out Twitter, or turn to his cadre of co-investors, who would probably have no trouble finding the money," says Andy Wu, an assistant professor at Harvard\'s business school. "The saying,\'If you owe the bank $100, that\'s your problem, but if you owe the bank $100 million, that\'s the bank\'s problem\' might apply here." If Twitter does file for bankruptcy, it could help Musk by allowing him to refinance the debt, which would make the company "more financially stable," 

In [13]:
print(tgt_text[0])

Elon Musk says Twitter bankruptcy is possible, but is that likely?<n>The billionaire has reportedly told staff that bankruptcy could be nigh if Twitter does not start making more money.<n>Musk cashed in some of his Tesla stock and saddled the social platform with $13 billion in debt.


In [ ]:
pip install -Iv protobuf==3.20.0

In [ ]:
!pip uninstall protobuf --yes

In [61]:
pegasus_df = pd.DataFrame(columns=["URL", "Summary"])

In [62]:
pegasus_df["URL"] = results_clean.keys()

In [63]:
pegasus_df["Summary"] = results_clean.values()

In [64]:
pegasus_df.to_csv("pegasus_test2.csv")